In [1]:
import pickle
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torchvision
import matplotlib.pyplot as plt
import sinabs
import sinabs.activation
import sinabs.layers as sl
from sinabs.from_torch import from_model
import os
import shutil
from sklearn.model_selection import KFold
print(torch.cuda.is_available())

True


In [2]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2080 Ti'

Data

In [3]:
class MyData(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.data_path = os.listdir(self.root_dir)
    
    def __getitem__(self, idx):
        data_name = self.data_path[idx]
        data_item_path = os.path.join(self.root_dir, data_name)
        with open(data_item_path, 'rb') as f:
            data = np.load(f)

        data = torch.from_numpy(data).float()
        data = torch.transpose(data, 0, 1)

        label = torch.tensor(eval(data_name[-5]), dtype=torch.long)
        
        return data, label

    def __len__(self):
        return len(self.data_path)

In [4]:
root_dir_1 = 'F:\Files\PhD\Braille\Data/braille-27letters-sphere/depth/depth-0.5mm/train'
root_dir_2 = 'F:\Files\PhD\Braille\Data/braille-27letters-sphere/depth/depth-0.5mm/test'

train_data = MyData(root_dir_1)
test_data  = MyData(root_dir_2)

# batch_size = 54
# train_loader = DataLoader(train_data, batch_size, shuffle=True)
# test_loader  = DataLoader(test_data, batch_size)

In [5]:
train_data[0][0].shape

torch.Size([700, 100])

Model

In [6]:
# linear_model = nn.Sequential(
#     nn.Linear(100, 400),
#     nn.ReLU(),
#     nn.Linear(400, 27)
# )
# linear_model = from_model(linear_model, batch_size=81, input_shape=(1, 650, 100), add_spiking_output=True)
# loss_fn = nn.CrossEntropyLoss()

Training

In [6]:
lr = 1e-4
device = 'cuda:0'

k_folds = 10
kfold = KFold(n_splits=k_folds, shuffle=False)
batch_size = 81
loss_fn = nn.CrossEntropyLoss()

fold_train_acc = []
fold_val_acc = []

for fold, (train_ids, val_ids) in enumerate(kfold.split(train_data)):
    # 模型初始化
    linear_model = nn.Sequential(
        nn.Linear(100, 400),
        nn.ReLU(),
        nn.Linear(400, 600),
        nn.ReLU(),
        nn.Linear(600, 200),
        nn.ReLU(),
        nn.Linear(200, 27)
    )
    linear_model = from_model(linear_model, batch_size=81, input_shape=(1, 650, 100), add_spiking_output=True)
    linear_model = linear_model.to(device)
    optimizer = torch.optim.Adam(linear_model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    # 分割数据集
    train_sub = Subset(train_data, train_ids)
    val_sub = Subset(train_data, val_ids)
    # 创建数据加载器
    train_loader = DataLoader(train_sub, batch_size, shuffle=True)
    val_loader  = DataLoader(val_sub, batch_size)

    # 训练模型
    linear_model.train()
    acc = 0
    epochs = 40
    for e in range(epochs):
        running_loss = 0.
        acc = 0
        scheduler.step()
        for i, (input, target) in enumerate(train_loader):
            optimizer.zero_grad()
            linear_model.reset_states()

            input = input.to(device)
            target = target.to(device)

            output = linear_model(input)
            sum_output = output.sum(1)
            loss = loss_fn(sum_output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss

            for j in range(batch_size):
                if sum_output[j].argmax() == target[j]:
                    acc += 1

        print("epoch: %d, accuracy: %.2f%%, running_loss: %.2f" % (e, acc/len(train_sub)*100, running_loss) )
    fold_train_acc.append(np.around(acc/len(train_sub)*100, 2))

    # 验证模型
    acc_num = 0
    for i, (data, target) in enumerate(val_loader):
        with torch.no_grad():
            linear_model.reset_states()
            data = data.to(device)
            target = target.to(device)
            output = linear_model(data)
            sum_output = output.sum(1)

        for j in range(batch_size):
            if sum_output[j].argmax() == target[j]:
                acc_num += 1
    print("accuracy on validation set: %.2f%%" % (acc_num/len(val_sub)*100))
    fold_val_acc.append(np.around(acc_num/len(val_sub)*100, 2))

f:\Softwares\Professional\Anaconda3\envs\synsense-gpu\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


epoch: 0, accuracy: 11.11%, running_loss: 148.31
epoch: 1, accuracy: 11.66%, running_loss: 147.81
epoch: 2, accuracy: 16.30%, running_loss: 138.34
epoch: 3, accuracy: 22.33%, running_loss: 127.21
epoch: 4, accuracy: 32.40%, running_loss: 109.47
epoch: 5, accuracy: 37.26%, running_loss: 100.90
epoch: 6, accuracy: 39.59%, running_loss: 98.87
epoch: 7, accuracy: 44.88%, running_loss: 91.78
epoch: 8, accuracy: 47.13%, running_loss: 86.99
epoch: 9, accuracy: 48.78%, running_loss: 84.34
epoch: 10, accuracy: 51.71%, running_loss: 81.78
epoch: 11, accuracy: 54.57%, running_loss: 80.63
epoch: 12, accuracy: 58.57%, running_loss: 72.56
epoch: 13, accuracy: 58.49%, running_loss: 74.63
epoch: 14, accuracy: 59.70%, running_loss: 79.89
epoch: 15, accuracy: 62.00%, running_loss: 70.71
epoch: 16, accuracy: 63.84%, running_loss: 70.22
epoch: 17, accuracy: 62.69%, running_loss: 79.47
epoch: 18, accuracy: 63.87%, running_loss: 73.39
epoch: 19, accuracy: 67.13%, running_loss: 60.98
epoch: 20, accuracy: 68.

In [10]:
print(fold_train_acc)
print(fold_val_acc)

sum = 0
cnt = 0
for i in fold_train_acc:
    sum += i
    cnt += 1
print(sum/cnt)

sum = 0
cnt = 0
for i in fold_val_acc:
    sum += i
    cnt += 1
print(sum/cnt)

[69.9, 70.26, 70.21, 72.02, 69.19, 69.77, 68.78, 70.53, 70.73, 68.81]
[68.15, 69.14, 70.62, 72.1, 65.43, 67.16, 71.6, 68.4, 68.4, 68.64]
70.02000000000001
68.964


In [ ]:
# linear_model.train()
# lr = 1e-5
# optimizer = torch.optim.Adam(linear_model.parameters(), lr)
# acc = 0

# epochs = 10
# for e in range(epochs):
#     running_loss = 0.
#     for i, (input, target) in enumerate(train_loader):
#         optimizer.zero_grad()
#         linear_model.reset_states()

#         output = linear_model(input)
#         sum_output = output.sum(1)
#         loss = loss_fn(sum_output, target)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss

#         for j in range(batch_size):
#             if sum_output[j].argmax() == target[j]:
#                 acc += 1

#     print("epoch: %d, accuracy: %.2f%%, running_loss: %.2f" % (e, acc/len(train_data)*100, running_loss) )
#     acc = 0

In [11]:
model_path = './models/train-27-sphere.pth'
torch.save(linear_model, model_path)

Testing

In [12]:
test_loader  = DataLoader(test_data, batch_size)

acc_num = 0
for i, (data, target) in enumerate(test_loader):
    with torch.no_grad():
        linear_model.reset_states()
        data = data.to(device)
        target = target.to(device)
        output = linear_model(data)
        sum_output = output.sum(1)

    for j in range(batch_size):
        if sum_output[j].argmax() == target[j]:
            acc_num += 1
print("accuracy on testing set: %.2f%%" % (acc_num/len(test_data)*100))

accuracy on testing set: 70.12%
